# Find the Nearest Hospitals

This projects aims at locating hospitals in densely populated cities such as New York, Tokyo, and Paris and compare it with it's population and analysig the city for pandemic situation